# Установка

Для установки - запустите все команды из раздела "Установка"

Также установку нужно запускать после старта/рестарта блокнота

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 
%cd /content/drive/My\ Drive/
!mkdir ru-gpts
%cd ru-gpts/

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

In [ ]:
!apt-get install llvm-9-dev

In [ ]:
!pip install cpufeature

In [ ]:
!pip install triton==0.2.3

In [ ]:
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7

In [ ]:
!ds_report

In [9]:
import deepspeed.ops.sparse_attention.sparse_attn_op

In [ ]:
!git clone https://github.com/nevstas/ru-gpts.git

In [ ]:
!pip install transformers==3.5.1

In [ ]:
!pip install natsort

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("ru-gpts/gw")
from generation_wrapper import RuGPT3XL
gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)

# Запуск генерации

Для того чтобы сгенерировать текст по ключу нужно заполнить файл "keywords.csv" - ключевыми словами и категорией WordPress, в которую будет добавлен пост. 

Скачайте файл "drive/MyDrive/ru-gpts/ru-gpts/keywords.csv", отредактируйте и залейте обратно.

Кол-во текстов будет столько сколько заданий в файле "keywords.csv"

Также в файле "keywords.csv" можно указать title будущего поста, но если не указать, то title сгенерируется из первых 60 символов поста

Затем запускаем код ниже, который в папке "drive/MyDrive/ru-gpts/ru-gpts/result/" будет создавать текста

In [ ]:
import os
import csv
import time
import glob
import json

script_path = "/content/drive/MyDrive/ru-gpts/ru-gpts"
max_count_attempt = 10
min_lenght_text = 300

def writeln(filename, str):
    global script_path
    with open(script_path + "//" + filename, 'w', encoding="utf-8") as the_file:
        the_file.write(str)

if not os.path.exists(script_path + '//result'):
    os.makedirs(script_path + '//result')
files = glob.glob('ru-gpts/result/*')
for f in files:
    os.remove(f)

print(time.strftime("%d.%m.%Y %H:%M:%S") + " Start")

with open(script_path + '/keywords.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    flag_first = False
    i = 0
    for row in spamreader:
        if not flag_first:
            flag_first = True
            continue
        i = i + 1
        keyword = row[0]
        text = ""
        count_attempt = 0
        continue_second = False
        while text == "":
            try:
                count_attempt = count_attempt + 1
                text = gpt.generate(
                    text=keyword,
                    do_sample=True,
                    max_length=5000,
                    no_repeat_ngram_size=3,
                    repetition_penalty=2.,
                )
                text = text[0]
                if len(text) < min_lenght_text:
                    print(time.strftime("%d.%m.%Y %H:%M:%S") + " result" + str(i) + ".txt Error: Lenght text less than " + str(min_lenght_text) + ". Skip this text/keyword")
                    continue_second = True
                    break
            except Exception as e:
                if count_attempt >= max_count_attempt:
                    print(time.strftime("%d.%m.%Y %H:%M:%S") + " result" + str(i) + ".txt Error: " + str(e) + " Count attempt: " + str(count_attempt) + ". Skip this text/keyword")
                    continue_second = True
                    break  
                print(time.strftime("%d.%m.%Y %H:%M:%S") + " result" + str(i) + ".txt Error: " + str(e) + " Count attempt: " + str(count_attempt) + ". Next attempt")
        if continue_second:
            continue
      
        text = text.replace("<|endoftext|>", "")
        text = os.linesep.join([s for s in text.splitlines() if s])
        params = {}
        params['category'] = row[1]                        
        if row[2]:
            params['title'] = row[2]
        text = "params=" + json.dumps(params) + "\n" + text
        writeln("result//result" + str(i) + ".txt", text)
        print(time.strftime("%d.%m.%Y %H:%M:%S") + " result" + str(i) + ".txt Done")



Экспорт в CSV

Результат будет в "drive/MyDrive/ru-gpts/ru-gpts/export.csv"

Скачиваем "export.csv", устанавливаем в WordPress плагин "WP All Import" и импортируем файл export.csv

In [2]:
!python3 ru-gpts/export_to_csv.py